In [59]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import re

In [3]:
class User:
    def __init__(self, tags) -> None:
        self.tags = tags

In [92]:
class TikTokScraper:
    def __init__(self) -> None:
        self.driver = None

    def create_and_run_scraper(self, options):
        self.driver = webdriver.Chrome(options=options)
        self.driver.get("https://www.tiktok.com/foryou")
        self.click_first_video()
    
    def quit_driver(self):
        self.driver.quit()

    def click_first_video(self):
        WebDriverWait(driver=self.driver, timeout=60).until(
            EC.element_to_be_clickable(
                (
                    By.XPATH,
                    "//*[@id=\"loginContainer\"]/div/div/div/div[2]"
                )
            )
        )
        # hide login (I don't know the consequences of this)
        self.driver.execute_script("document.evaluate('/html/body/div[5]', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.style.display='none';")
        WebDriverWait(driver=self.driver, timeout=60).until(
            EC.element_to_be_clickable(
                (
                    By.XPATH,
                    '//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/div/div[2]/div[1]',
                )
            )
        ).click()
        WebDriverWait(self.driver, 60).until(
            EC.element_to_be_clickable(
                (
                    By.XPATH,
                    "//*[@id=\"tiktok-verify-ele\"]/div/div[4]/div/a[1]"
                )
            )
        )
        # hide captcha (I don't know the consequences of this)
        self.driver.execute_script("document.getElementById('tiktok-verify-ele').style.display='none';")

    # Gets and returns tags associated with the current video
    def get_current_video_tags(driver):
        # wait until comment section is loaded
        WebDriverWait(driver, timeout=60).until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="app"]/div[2]/div[3]/div[2]/div[2]/div[1]')
            )
        )

        comment_parent_element = driver.find_element(
            By.XPATH, '//*[@id="app"]/div[2]/div[3]/div[2]/div[2]/div[1]'
        )
        comment_parent_children = comment_parent_element.find_elements(By.TAG_NAME, "a")
        comment_categories = [
            child.get_attribute("href") for child in comment_parent_children
        ]
        tags = []
        for category in comment_categories:
            if "tag" in category:
                tag = category.split("/")[-1]
                tags.append(tag)

        return tags

    def watch_n_minutes(low, high):
        return np.random.randint(low=low, high=high)

    def watch_n_videos(self, n, user, tags):
        for i in range(n):
            watch_or_skip = watch_or_skip(self.driver, user, tags)
            if watch_or_skip:
                pass
            time.sleep(5)
            self.next_video()

    def get_video_length(driver):
        video_length = str(
            WebDriverWait(driver=driver, timeout=60)
            .until(
                EC.presence_of_element_located(
                    (
                        By.XPATH,
                        '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[2]',
                    )
                )
            )
            .get_attribute("innerHTML")
        )
        video_length = video_length[video_length.index("/") + 1 : len(video_length)]
        if video_length[0:1] == "0":
            video_length = video_length[1 : len(video_length)]
        video_minutes = video_length[0 : video_length.index(":")]
        video_seconds = video_length[video_length.index(":") + 1 : len(video_length)]
        video_length_in_seconds = int(video_minutes) * 60 + int(video_seconds)
        return video_length_in_seconds

In [89]:
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument("--disable-infobars")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_argument("--headless=new")

In [62]:
user = User(tags=["cats", "dogs", "animals"])

In [90]:
scraper = TikTokScraper()
scraper.create_and_run_scraper(options=options)

In [91]:
scraper.quit_driver()